### Cell 1: Imports & Configuration

In [1]:
import os, json, time, random, pathlib, pickle
from pathlib import Path
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import faiss
import gradio as gr

# --- Main Configuration ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_DIR = Path("./data")
CACHE_DIR = Path("./cache")
CACHE_DIR.mkdir(parents=True, exist_ok=True)

# --- Tuning Knobs ---
CLASSES_TO_USE = None
IMAGES_PER_CLASS = 60
TOP_K = 6
BATCH_SIZE = 64
EMB_CACHE_NAME = "food101_clip_ViT-B-32_subset"
MODEL_NAME = "openai/clip-vit-base-patch32"

# --- Reproducibility ---
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

print("✅ Cell 1: Imports and configuration loaded.")

/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Cell 1: Imports and configuration loaded.


### Cell 2: Load Dataset

In [2]:
DATA_DIR.mkdir(parents=True, exist_ok=True)
train_ds = datasets.Food101(root=str(DATA_DIR), download=True, split='train')

class_to_idx = train_ds.class_to_idx
idx_to_class = {v:k for k,v in class_to_idx.items()}

print(f"✅ Cell 2: Food-101 dataset loaded. Found {len(train_ds)} images.")

✅ Cell 2: Food-101 dataset loaded. Found 75750 images.


### Cell 3: Create or Load Cached Image Subset

In [3]:
# Define the path for our cache file
subset_cache_path = CACHE_DIR / "subset_data.pkl"

# Caching logic: Check if the pre-computed subset file exists
if subset_cache_path.exists():
    print(f"✅ Loading pre-computed subset from cache: {subset_cache_path}")
    with open(subset_cache_path, 'rb') as f:
        cached_data = pickle.load(f)
    subset_indices = cached_data['subset_indices']
    
else:
    print("Cache not found. Building subset from scratch. This may take a minute...")
    if CLASSES_TO_USE is not None:
        keep_idx = {class_to_idx[c] for c in CLASSES_TO_USE if c in class_to_idx}
    else:
        keep_idx = set(range(len(class_to_idx)))

    all_labels = [label for _, label in tqdm(train_ds, desc="Extracting labels")]

    per_class_counter = {i:0 for i in keep_idx}
    subset_indices = []
    for idx, label in enumerate(all_labels):
        if label in keep_idx and per_class_counter[label] < IMAGES_PER_CLASS:
            subset_indices.append(idx)
            per_class_counter[label] += 1
        if all(per_class_counter[i] >= IMAGES_PER_CLASS for i in keep_idx):
            break
            
    print(f"✅ Subset created. Caching result to {subset_cache_path}")
    with open(subset_cache_path, 'wb') as f:
        pickle.dump({'subset_indices': subset_indices}, f)

print(f"✅ Cell 3: Subset ready with {len(subset_indices)} images.")

✅ Loading pre-computed subset from cache: cache/subset_data.pkl
✅ Cell 3: Subset ready with 6060 images.


### Cell 4: Load CLIP Model & Processor

In [4]:
print("Loading CLIP model and processor...")
model = CLIPModel.from_pretrained(MODEL_NAME, use_safetensors=True).to(DEVICE).eval()
processor = CLIPProcessor.from_pretrained(MODEL_NAME)
print("✅ Cell 4: CLIP model loaded.")

Loading CLIP model and processor...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ Cell 4: CLIP model loaded.


### Cell 5: Build or Load Image Embedding Index

In [5]:
emb_cache_base = CACHE_DIR / EMB_CACHE_NAME
img_emb_path = emb_cache_base.with_suffix(".npy")
meta_path    = emb_cache_base.with_suffix(".meta.json")
faiss_path   = emb_cache_base.with_suffix(".faiss")

def collate_fn(batch):
    images = [b[0] for b in batch]
    labels = [b[1] for b in batch]
    return images, labels

def compute_image_embeddings():
    subset_for_loader = torch.utils.data.Subset(train_ds, subset_indices)
    loader = DataLoader(subset_for_loader, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=collate_fn)
    
    all_embs = []
    all_meta = []
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Embedding images"):
            # This is the critical fix for the recurring error.
            inputs = processor(images=images, return_tensors="pt", padding=True).to(DEVICE)
            
            image_emb = model.get_image_features(**inputs)
            image_emb /= image_emb.norm(p=2, dim=-1, keepdim=True)
            all_embs.append(image_emb.detach().cpu().numpy())
            for lab in labels:
                all_meta.append({"label_idx": int(lab), "label": idx_to_class[int(lab)]})
    return np.vstack(all_embs).astype("float32"), all_meta

if img_emb_path.exists() and meta_path.exists() and faiss_path.exists():
    print("Loading cached embeddings & index...")
    img_embs = np.load(img_emb_path)
    with open(meta_path, "r", encoding="utf-8") as f:
        meta = json.load(f)
    index = faiss.read_index(str(faiss_path))
else:
    print("No cache found. Computing embeddings from scratch...")
    img_embs, meta = compute_image_embeddings()
    np.save(img_emb_path, img_embs)
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)
    dim = img_embs.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(img_embs)
    faiss.write_index(index, str(faiss_path))

print(f"✅ Cell 5: Index ready. Contains {index.ntotal} vectors.")

Loading cached embeddings & index...
✅ Cell 5: Index ready. Contains 6060 vectors.


### Cell 6: Map Subset Indices to File Paths

In [16]:
# %%
if not hasattr(train_ds, "_image_files"):
    raise AttributeError("Your version of torchvision's Food101 is missing the `_image_files` attribute.")

# FINAL FIX: The `_image_files` list already contains the full relative path.
# We will use these paths directly without prepending any root directory.
all_image_paths = [Path(p) for p in train_ds._image_files]

# Select only the paths for the images that are in our created subset
subset_image_paths = [all_image_paths[i] for i in subset_indices]

print(f"✅ Cell 6: Corrected {len(subset_image_paths)} file paths for the subset.")

✅ Cell 6: Corrected 6060 file paths for the subset.


### Cell 7: Define Search Logic

In [17]:
# %%
def text_to_embedding(query: str) -> np.ndarray:
    with torch.no_grad():
        inputs = processor(text=[query], return_tensors="pt", padding=True).to(DEVICE)
        text_emb = model.get_text_features(**inputs)
        text_emb /= text_emb.norm(p=2, dim=-1, keepdim=True)
    return text_emb.detach().cpu().numpy().astype("float32")

def search_images(query: str, k: int = TOP_K):
    if not query.strip(): return []
    
    q_emb = text_to_embedding(query)
    sims, idxs = index.search(q_emb, k)
    
    # --- Start of Debugging Block ---
    print("\n--- DEBUGGING FILE PATHS ---")
    results = []
    for rank, (i, s) in enumerate(zip(idxs[0], sims[0]), start=1):
        img_path = subset_image_paths[i]
        label = meta[i]["label"]
        caption = f"#{rank}: {label.replace('_',' ')} (Score: {s:.3f})"
        
        # We'll print the path and check if the file exists
        print(f"Path for rank #{rank}: {img_path}")
        print(f"Does file exist? -> {img_path.exists()}")
        
        results.append((str(img_path), caption))
    print("--------------------------\n")
    # --- End of Debugging Block ---

    return results

# --- Quick smoke test ---
test_results = search_images("pizza with cheese")
print(f"✅ Cell 7: Search functions defined. Test search completed.")


--- DEBUGGING FILE PATHS ---
Path for rank #1: data/food-101/images/apple_pie/1111062.jpg
Does file exist? -> True
Path for rank #2: data/food-101/images/pizza/1202925.jpg
Does file exist? -> True
Path for rank #3: data/food-101/images/pizza/1029698.jpg
Does file exist? -> True
Path for rank #4: data/food-101/images/pizza/1097980.jpg
Does file exist? -> True
Path for rank #5: data/food-101/images/pizza/1054420.jpg
Does file exist? -> True
Path for rank #6: data/food-101/images/caesar_salad/1167058.jpg
Does file exist? -> True
--------------------------

✅ Cell 7: Search functions defined. Test search completed.


### Cell 8: Launch the Gradio User Interface

In [18]:
# %%
EXAMPLES = [
    "pizza margherita",
    "chocolate cake",
    "sushi roll with salmon",
    "chicken curry",
    "greek salad",
    "ramen noodles",
    "hamburger with fries",
]

with gr.Blocks(title="Food Dish Image Search", theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🍔 Food Dish Image Search 🍣")
    gr.Markdown("Enter a dish name (e.g., 'spaghetti bolognese') and see the closest matching images from the Food-101 dataset.")
    with gr.Row():
        with gr.Column(scale=2):
            query = gr.Textbox(label="Dish Description", placeholder="e.g., 'spaghetti bolognese'")
            k = gr.Slider(1, 12, value=TOP_K, step=1, label="Number of Images to Find")
            btn = gr.Button("Search", variant="primary")
        with gr.Column(scale=3):
             # Simplified for better compatibility
             gallery = gr.Gallery(label="Results", columns=3)
    
    btn.click(search_images, inputs=[query, k], outputs=gallery)
    gr.Examples(EXAMPLES, inputs=[query])

# CORRECTED LINE: Set share=True to create a public link.
#demo.launch(share=True)
# FINAL FIX: Use inline=True to embed the app directly in the notebook output.
demo.launch(inline=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.



--- DEBUGGING FILE PATHS ---
Path for rank #1: data/food-101/images/chocolate_cake/1077967.jpg
Does file exist? -> True
Path for rank #2: data/food-101/images/chocolate_cake/1169806.jpg
Does file exist? -> True
Path for rank #3: data/food-101/images/chocolate_cake/1001084.jpg
Does file exist? -> True
--------------------------


--- DEBUGGING FILE PATHS ---
Path for rank #1: data/food-101/images/filet_mignon/11785.jpg
Does file exist? -> True
Path for rank #2: data/food-101/images/filet_mignon/1236710.jpg
Does file exist? -> True
Path for rank #3: data/food-101/images/filet_mignon/1001477.jpg
Does file exist? -> True
--------------------------


--- DEBUGGING FILE PATHS ---
Path for rank #1: data/food-101/images/filet_mignon/1007877.jpg
Does file exist? -> True
Path for rank #2: data/food-101/images/steak/114601.jpg
Does file exist? -> True
Path for rank #3: data/food-101/images/filet_mignon/11785.jpg
Does file exist? -> True
--------------------------

